# Environment setting
環境設定

In [1]:
import os
import requests
import sys
from pathlib import Path
from pprint import pprint
from typing import Optional

import pandas as pd


# determine branch, default is main
branch: str = "main"

# determine subfolder, default is None (petsard/demo/)
subfolder: Optional[str] = "use-cases"


# Check if running in Google Colab, if so, download the utils.py file from GitHub
is_colab: bool = "COLAB_GPU" in os.environ
if is_colab:
    utils_url = (
        f"https://raw.githubusercontent.com/nics-tw/petsard/{branch}/demo/utils.py"
    )
    response = requests.get(utils_url)

    if response.status_code == 200:
        with open("utils.py", "w") as f:
            f.write(response.text)

        Path("__init__.py").touch()
    else:
        raise RuntimeError(
            f"Failed to download utils.py. Status code: {response.status_code}"
        )


# If not colab, and also contains subfolderl, add the correct path of util.py
else:
    if subfolder:
        sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
# Now import and run the setup
from utils import (
    get_yaml_path,
    setup_environment,
)


setup_environment(
    is_colab,
    branch,
    benchmark_data=[
        "adult-income",
    ],
    example_files=[
        "demo/custom-evaluation.py",
    ],
    subfolder=subfolder,
)

In [3]:
from petsard import Executor

# YAML Configuration for PETsARD
PETsARD 的 YAML 設定

## Custom Evaluation
自定義評測

In [4]:
yaml_file_case: str = "custom-synthesis.yaml"

yaml_path_case: str = get_yaml_path(
    is_colab=is_colab,
    yaml_file=yaml_file_case,
    branch=branch,
    subfolder=subfolder,
)

Configuration content:
---
Loader:
  data:
    filepath: 'benchmark://adult-income'
Preprocessor:
  demo:
    method: 'default'
Synthesizer:
  custom:
    method: 'custom_method'
    module_path: 'custom-synthesis.py'  # Path to your custom synthesizer
    class_name: 'MySynthesizer_Shuffle'  # Synthesizer class name
Postprocessor:
  demo:
    method: 'default'
Evaluator:
  demo:
    method: 'default'
Reporter:
  save_report_global:
    method: 'save_report'
    granularity: 'global'
...


### Execution and Result
執行與結果

In [5]:
# Initialize and run executor
exec_case = Executor(config=yaml_path_case)
exec_case.run()

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 108.29it/s]|
Column Shapes Score: 95.5%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:00<00:00, 372.25it/s]|
Column Pair Trends Score: 59.01%

Overall Score (Average): 77.25%

Now is petsard[Report]_[global] save to csv...


In [6]:
exec_case.get_result()[
    "Loader[data]_Preprocessor[demo]_Synthesizer[custom]_Postprocessor[demo]_Evaluator[demo]_Reporter[save_report_global]"
]["[global]"]

,full_expt_name,Loader,Preprocessor,Synthesizer,Postprocessor,Evaluator,demo_Score,demo_Column Shapes,demo_Column Pair Trends
result,Loader[data]_Preprocessor[demo]_Synthesizer[cu...,data,demo,custom,demo,[global],0.77,NaN,NaN
